In [3]:
import json
from pathlib import Path

In [53]:
JSONDIR = "/tmp/reassemble/"
TSVDIR = "/tmp/word-tsv"
TXTDIR = "/Users/joregan/Playing/audio_clips/"

In [54]:
JSONPATH = Path(JSONDIR)
TSVPATH = Path(TSVDIR)
TXTPATH = Path(TXTDIR)

In [35]:
if not TSVPATH.is_dir():
    TSVPATH.mkdir()

In [55]:
def read_json(filename):
    if type(filename) is Path:
        filename = str(filename)
    out = []
    with open(filename) as inf:
        data = json.load(inf)
    for seg in data["segments"]:
        out.append({
            "start": seg["start"],
            "end": seg["end"],
            "word": seg["text"]
        })
    return out

In [56]:
def reinterpret_json(filename: Path):
    data = read_json(filename)
    stem = filename.stem
    orig_text = TXTPATH / f"{stem}.txt"
    json_words = [x["word"] for x in data]
    json_txt = " ".join(json_words)
    with open(str(orig_text)) as inf:
        text = inf.read().strip()
    if json_txt != text:
        print(text, json_txt)
    

In [58]:
reinterpret_json(Path("/tmp/reassemble/hsi_7_0719_227_003_main_490060_501574.json"))

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/reassemble/hsi_7_0719_227_003_main_490060_501574.json'

In [46]:
filepieces = {}
for file in JSONPATH.glob("*.json"):
    filestem = file.stem
    if not filestem.startswith("hsi_"):
        continue
    pieces = filestem.split("_")
    if len(pieces) != 8:
        continue
    base = "_".join(pieces[0:6])
    if not base in filepieces:
        filepieces[base] = []

    startint = int(pieces[6])
    endint = int(pieces[7])
    words = read_json(file)
    for word in words:
        w = word["word"].strip()
        if w.startswith("[") or w.endswith("]"):
            continue
        start = (float(startint) / 1000.0) + word["start"]
        end = (float(startint) / 1000.0) + word["end"]
        filepieces[base].append((start, end, w))

In [51]:
for piece in filepieces:
    pieces = filepieces[piece]
    pieces_sorted = sorted(pieces, key=lambda x: x[0])
    outfile = TSVPATH / f"{piece}.tsv"
    with open(outfile, "w") as of:
        for item in pieces_sorted:
            of.write(f"{item[0]:.03f}\t{item[1]:.03f}\t{item[2]}\n")